In [11]:
%reload_ext autoreload 
%autoreload 2

In [9]:
HOPSWORKS_PROJECT_NAME = 'Bike_sharing_Demand_ML'

In [10]:
import os 
from dotenv import load_dotenv 
from src.paths import PARENT_DIR

load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']



In [2]:
from datetime import datetime 
import pandas as pd 
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print('Downloading data from {} to {}'.format(from_year, to_year))

rides = pd.DataFrame()

for year in range(from_year, to_year + 1):
    #Download data for the whole year
    rides_one_year = load_raw_data(year)

    #appending the rows
    rides = pd.concat([rides, rides_one_year])




File 2022 was already in local storage
File 2023 was already in local storage
File 2024 was already in local storage


In [3]:
print(f'length of rides: {len(rides)}')

length of rides: 7073863


In [4]:
rides.pickup_datetime.max()

Timestamp('2024-06-30 23:59:41')

In [5]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 373/373 [00:30<00:00, 12.30it/s]


In [6]:
ts_data.tail(740)

,pickup_hour,rides,pickup_location_id
7244188,2024-05-31 04:00:00,0,57
7244189,2024-05-31 05:00:00,1,57
7244190,2024-05-31 06:00:00,0,57
7244191,2024-05-31 07:00:00,0,57
7244192,2024-05-31 08:00:00,1,57
...,...,...,...
7244923,2024-06-30 19:00:00,0,57
7244924,2024-06-30 20:00:00,0,57
7244925,2024-06-30 21:00:00,0,57
7244926,2024-06-30 22:00:00,0,57


In [7]:
import hopsworks
hopsworks.__version__

'3.7.0'

In [12]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/881813


In [13]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [14]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [15]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [17]:
feature_group.insert(ts_data,  write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/881813/fs/877636/fg/1077584


Uploading Dataframe: 0.00% |          | Rows 0/7244928 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/881813/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x261fa57bc10>, None)